# Partie I clustering des documents de la décennie 1960 

### Importation des Bibliothèques

In [ ]:
import collections
import os
import string
import sys

import pandas as pd
from nltk import word_tokenize
from nltk.corpus import stopwords
from pprint import pprint
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cosine
import nltk

nltk.download('punkt')

In [ ]:
data_path = "../data/txt/"

### selection décennie 1960s

In [ ]:
DECADE = '1960' # on prend tous les doc des année 1960s

### Chargement de tous les fichiers de la décennie 60s et en création d'une liste de textes

In [ ]:
files = [f for f in sorted(os.listdir(data_path)) if f"_{DECADE[:-1]}" in f]

In [ ]:
# Exemple de fichiers
files[:5] #afin de me rassuserer que tous les doc de ma decennie on été selectionner.

In [ ]:
texts = [open(data_path + f, "r", encoding="utf-8").read() for f in files]

In [ ]:
# Exemple de textes
texts[0][:400]

### Vectoriser ( transformé en chiffre) les fichiers à l'aide de TF-IDF

In [ ]:
# Création d'une fonction de pré-traitement (enlevé les pontuation, mot long, etc...)
def preprocessing(text, stem=True):
    """ Tokenize text and remove punctuation """
    text = text.translate(string.punctuation)
    tokens = word_tokenize(text)
    return tokens

### Instancier le modèle TF-IDF avec ses arguments

In [ ]:
vectorizer = TfidfVectorizer(
    tokenizer=preprocessing,
    stop_words=stopwords.words('french'),
    max_df=0.5,
    min_df=0.1,
    lowercase=True)

### Construction de la matrice de vecteurs à l'aide de la fonction `fit_transform`

In [ ]:
tfidf_vectors = vectorizer.fit_transform(texts)

In [ ]:
# Détail de la matrice
tfidf_vectors

### Impresion du vecteur tf-IDF du premier document

In [ ]:
pd.Series(
    tfidf_vectors[0].toarray()[0],
    index=vectorizer.get_feature_names_out()
    ).sort_values(ascending=False)

In [ ]:
### 

In [ ]:
cosine([1, 2, 3], [1, 2, 3]) # l'idée plus les text son proche plus il on les meme mot plus la distance son proche

In [ ]:
cosine([1, 2, 3], [1, 2, 2])

In [ ]:
cosine([1, 2, 3], [2, 2, 2])

### Tests sur nos fichiers

In [ ]:
tfidf_array = tfidf_vectors.toarray()

In [ ]:
# Vecteur du document 0
tfidf_array[0]

In [ ]:
# Vecteur du document 1
tfidf_array[1]

In [ ]:
cosine(tfidf_array[0], tfidf_array[1])

### Application du clustering sur les vecteurs TF-IDF des documents¶

In [ ]:
# Définition du nombre de clusters
N_CLUSTERS = 3 #nous choisisons d'enfaire 3 groupes car le resultat nous semble permtinant

In [ ]:
# Instancier le modèle K-Means et ses arguments
km_model = KMeans(n_clusters=N_CLUSTERS)

In [ ]:
clusters = km_model.fit_predict(tfidf_vectors)

In [ ]:
clustering = collections.defaultdict(list)

for idx, label in enumerate(clusters):
    clustering[label].append(files[idx])

In [ ]:
pprint(dict(clustering))

In [ ]:
pca = PCA(n_components=2)
reduced_vectors = pca.fit_transform(tfidf_vectors.toarray())

In [ ]:
reduced_vectors[:10]

### Générer le plot

In [ ]:
x_axis = reduced_vectors[:, 0]
y_axis = reduced_vectors[:, 1]

plt.figure(figsize=(10,10))
scatter = plt.scatter(x_axis, y_axis, s=100, c=clusters)

# Ajouter les centroïdes
centroids = pca.transform(km_model.cluster_centers_)
plt.scatter(centroids[:, 0], centroids[:, 1],  marker = "x", s=100, linewidths = 2, color='red')

# Ajouter la légende
plt.legend(handles=scatter.legend_elements()[0], labels=set(clusters), title="Clusters")

# Partie II Entraînement d'un modèle word2vec et Exploration grace aux fonctions similarity et most_similar

#### importation des libraries necessaires

In [ ]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

###  Création d'un objet qui streame les lignes d'un fichier pour économiser de la RAM

In [ ]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [ ]:
infile = f"../data/sents.txt"
sentences = MySentences(infile)

In [ ]:
# Détection des bigrams
bigram_phrases = Phrases(sentences)

In [ ]:
type(bigram_phrases.vocab)

In [ ]:
len(bigram_phrases.vocab.keys())

In [ ]:
key_ = list(bigram_phrases.vocab.keys())[144]
print(key_)

In [ ]:
bigram_phrases.vocab[key_]

In [ ]:
# onversion des Phrases en objet Phraser
bigram_phraser = Phraser(phrases_model=bigram_phrases)

In [ ]:
# Extraction des trigrams
trigram_phrases = Phrases(bigram_phraser[sentences])

In [ ]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

In [ ]:
# Création d'un corpus d'unigrams, bigrams, trigrams
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [ ]:
print(corpus[:100])

### Entrainement de notre modèle Word2Vec sur ce corpus

In [ ]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=7, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=10, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus (jouer sur la frequence des mot)
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.(plus on a en fait plus ca prend du temps)
)

In [ ]:
# sauvegarde du modele
outfile = f"../data/newspapers.model"
model.save(outfile)

### Exploration  du modèle

In [ ]:
# hargement du modèle
model = Word2Vec.load("../data/newspapers.model")

In [ ]:
model.wv["ministre"] # par exemple on imprime le vecteur du mot ministre

### similarité entre deux termes

In [ ]:
model.wv.similarity("ministre", "delegue")

In [ ]:
model.wv.similarity("france", "congo")

In [ ]:
model.wv.similarity("belgique", "france")

### chercher les mots les plus proches d'un terme donné

In [ ]:
model.wv.most_similar("president", topn=10)

In [ ]:
model.wv.most_similar("gouvernement", topn=10)

In [ ]:
model.wv.most_similar("avocat", topn=10)

### recherches complexes à travers l'espace vectoriel

In [ ]:
print(model.wv.most_similar(positive=['paris', 'londres'], negative=['guinee'])) 